## Problem Description

Imagine that you are a thief breaking into a house. There are so many valuables to steal, but you’re just one person who can only carry so much. Each item has a weight and value, and your goal is to maximize the total value of items while remaining within the weight limit of your knapsack. Create a knapsack() function that takes in:

the total amount of weight you can carry
an array of the weights of all of the items
an array of the values of all of the items
and returns the maximum value that you will be able to carry.

For example, let’s say your knapsack can carry 10 units of weight. The item weights are [3, 6, 8] and their values are [50, 60, 100]. Your knapsack function should return 110 since you can carry the first and second items, whose values total 110. If you tried to carry the third item, which has the value of 100, you wouldn’t be able to fit anything else in the knapsack.

## Strategy


For this task I would like to use tabulation to solve the knapsack subproblem for the sublists of the first i elements. If item_list is the list of items, this would solve the subproblem for each sublist of the form item_list[:i], with i starting at 1. But there is one subtlety: to get to the next step, we need to add the weight of the next item in the list to all of the previous lists, which may or may not go over the weight limit. Since the weight limit is constantant, we need to solve each of these subproblems for all of the weights up to the weight limit. Or, I guess, we just need to make sure the weight limit isn't too big! Oops. Well, that was easier than I thought...

For implementation, it might be easiest to keep track of everything using indices, since we're given a list of weights and a list of values. We might as well keep track of these items by their index number since that's probably easier to store than items or even tuples of items. 

Below is my initial attempt with the greedy algorithm. This time around, we (hopefully) won't be overlooking possible arrangements of things in the sack. 

In [ ]:
def knapsack(weight_cap, weights, values):
  def division_bandaid(a,b):
    if b == 0:
      return 10**40
    else: 
      return a/b
  weight_densities = [division_bandaid(value,weight) for value, weight in zip(values, weights)]
  weight_density_tuples = list(enumerate(weight_densities))
  weight_density_tuples.sort(key = lambda pair: pair[1])
  density_ordering = [a for a, b in weight_density_tuples]
  print(density_ordering)
  carried_weight, carried_value = 0, 0
  for index in density_ordering:
    carried_weight += weights[index]
    if carried_weight > weight_cap:
      break
    carried_value += values[index]
  return carried_value

weight_cap = 10
weights = [3, 6, 8]
values = [50, 60, 100]
print(knapsack(weight_cap, weights, values))

Time to start working! 

## My First Attempt Using Tabulation

In [10]:
def knapsack(weight_cap, weights, values):
    possible_packing_list = []
    for new_index, (new_weight, new_value) in enumerate(zip(weights, values)):
    # For each index in the list, we're going to add tuples of the form (carried_index_list, weight)
    #Creating single-value lists to simulate the thief possibly grabbing the index item as his first item. 
        if new_weight <= weight_cap:
            possible_packing_list.append(([new_index], new_weight, new_value))
        # Appending each part of the tuple if the weight would not go over the limit. 
        for (current_index_list, current_total_weight, current_total_value) in possible_packing_list:
            if current_total_weight + new_weight <= weight_cap:
                possible_packing_list.append(
                    (current_index_list + [new_index], current_total_weight + new_weight, current_total_value + new_value)
                )
        possible_total_values_list = [carried_item_tuple[2] for carried_item_tuple in possible_packing_list]
    print(f"Possible values: {possible_total_values_list}")
    if possible_total_values_list == []:
        return 0
    return max(possible_total_values_list)







weight_cap = 10
weights = [3, 6, 8]
values = [50, 60, 100]
print(knapsack(weight_cap, weights, values))

Possible values: [50, 100, 150, 60, 110, 100]
150


The simple test below worked! Yes! 

In [ ]:
def knapsack(weight_cap, weights, values):
    possible_packing_list = []
    for new_index, (new_weight, new_value) in enumerate(zip(weights, values)):
    # For each index in the list, we're going to add tuples of the form (carried_index_list, weight)
    #Creating single-value lists to simulate the thief possibly grabbing the index item as his first item. 
        if new_weight <= weight_cap:
            possible_packing_list.append(([new_index], new_weight, new_value))
        # Appending each part of the tuple if the weight would not go over the limit. 
        for (current_index_list, current_total_weight, current_total_value) in possible_packing_list:
            if current_total_weight + new_weight <= weight_cap:
                possible_packing_list.append(
                    (current_index_list + [new_index], current_total_weight + new_weight, current_total_value + new_value)
                )
        possible_total_values_list = [carried_item_tuple[2] for carried_item_tuple in possible_packing_list]
    print(f"Possible values: {possible_total_values_list}")
    if possible_total_values_list == []:
        return 0
    return max(possible_total_values_list)







weight_cap = 1000
weights = [2*i for i in range(10000)]
values = [10*i for i in range(10000)]
print(knapsack(weight_cap, weights, values))

## Test Failed! 

But when I "tested" it with the Codecademy website, the test failed, giving the message below. 

"Tests failed to run due to an error: "Failed to test your code.". Check your code and try again."

I like running things in this program, so I transferred over the code to something below and tried some bigger numbers. After about 10 seconds, it was fairly clear that this was too slow. Luckily, I realized that keeping track of all the intermediate lists is completely unecessary. We just need to store the total weight and total value parameters. 

## Let's try again without storing the lists. 

In [5]:
def knapsack(weight_cap, weights, values):
    possible_packing_list = []
    for new_index, (new_weight, new_value) in enumerate(zip(weights, values)):
    # For each index in the list, we're going to add tuples of the form (carried_index_list, weight)
    #Creating single-value lists to simulate the thief possibly grabbing the index item as his first item. 
        for (current_total_weight, current_total_value) in possible_packing_list:
            if current_total_weight + new_weight <= weight_cap:
                possible_packing_list.append(
                    (current_total_weight + new_weight, current_total_value + new_value)
                )
        possible_total_values_list = [carried_item_tuple[1] for carried_item_tuple in possible_packing_list]
        if new_weight <= weight_cap:
            possible_packing_list.append((new_weight, new_value))
        
    print(f"Possible values: {possible_total_values_list}")
    if possible_total_values_list == []:
        return 0
    return max(possible_total_values_list)







weight_cap = 10
weights = [3, 6, 8, 5, 5, 9]
values = [50, 60, 100, 50, 50, 95]
print(knapsack(weight_cap, weights, values))

Possible values: [50, 110, 60, 100, 100, 50, 100, 100, 50]
110


Still very slow! Let's try a dictionary instead, where the keys are the weights and the values are, well, the values! If a key:value pair is already there with a lower value, we'll replace it with the higher value. 

In [22]:
def knapsack(weight_cap, weights, values):
    possible_payloads = {i:0 for i in range(weight_cap + 1)}
    for new_weight, new_value in zip(weights, values):
        for current_total_weight, current_total_value in possible_payloads.items():
            new_total_weight, new_total_value = current_total_weight + new_weight, current_total_value + new_value
            if possible_payloads.get(new_total_weight) == None:
                if new_total_weight >= weight_cap:
                    break
                possible_payloads[new_total_weight] = new_total_value
            elif new_total_value >= possible_payloads[new_total_weight]:
                possible_payloads[new_total_weight] = new_total_value
    possible_total_values_list = [value for key, value in possible_payloads.items()]
    print(f"Possible values: {possible_total_values_list}")
    return max(possible_total_values_list)









weight_cap = 1000
weights = [2*i for i in range(1000)]
values = [10*i for i in range(1000)]
print(knapsack(weight_cap, weights, values))

Possible values: [0, 0, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80, 80, 90, 90, 100, 100, 110, 110, 120, 120, 130, 130, 140, 140, 150, 150, 160, 160, 170, 170, 180, 180, 190, 190, 200, 200, 210, 210, 220, 220, 230, 230, 240, 240, 250, 250, 260, 260, 270, 270, 280, 280, 290, 290, 300, 300, 310, 310, 320, 320, 330, 330, 340, 340, 350, 350, 360, 360, 370, 370, 380, 380, 390, 390, 400, 400, 410, 410, 420, 420, 430, 430, 440, 440, 450, 450, 460, 460, 470, 470, 480, 480, 490, 490, 500, 500, 510, 510, 520, 520, 530, 530, 540, 540, 550, 550, 560, 560, 570, 570, 580, 580, 590, 590, 600, 600, 610, 610, 620, 620, 630, 630, 640, 640, 650, 650, 660, 660, 670, 670, 680, 680, 690, 690, 700, 700, 710, 710, 720, 720, 730, 730, 740, 740, 750, 750, 760, 760, 770, 770, 780, 780, 790, 790, 800, 800, 810, 810, 820, 820, 830, 830, 840, 840, 850, 850, 860, 860, 870, 870, 880, 880, 890, 890, 900, 900, 910, 910, 920, 920, 930, 930, 940, 940, 950, 950, 960, 960, 970, 970, 980, 980, 990, 990, 1000